In [ ]:
SPECIFIC_COLUMNS = ["대학코드", "대학명", "국가", "review_link", "gen_info", "env_info", "food_info", "study_info", "office_info", "facil_info", "mhct_info","help_info","etc_info"]

In [20]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

# display all rows and columns when dataframe is printed, max columnwidth(=legnth of printed cell) as 500 letters
pd.set_option("display.max_rows", None, "display.max_columns", None, "display.max_colwidth", 500)
np.random.seed(0)

from konlpy.tag import Okt
twitter = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in twitter.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [21]:
SOURCE = "./data/univ_text_data"
SOURCE_ABSTRACT = "./data/abstract"
SOURCE_SPECIFIC = "./data/specific"
SOURCE_BERT = "data_sentiment/abstract_bert"

In [22]:
univ_code = "DK000003"

In [30]:
df_sample = pd.read_csv(f"{SOURCE_SPECIFIC}/{univ_code}_reviews_specific.csv", encoding="utf-8")
df_sample.sample(5)

,gen_info,env_info,food_info,study_info,office_info,facil_info,mhct_info,help_info,etc_info
44,"코펜하겐 대학교는 하나의 캠퍼스가 통합되어 있는 일반적인 한국식 학교와 달리 도시 곳곳에 대학 건물이 산재되어 있는 형식입니다. 사회과학대(CSS), 인문대(KUA), 생명과학대, 자연과학대 등 4,5개의 캠퍼스로 구성되어있었는데 경제학과를 다닌 저는 사회과학대에서 수업을 들었습니다. 사회과학대인 CSS는 도시 중심에 위치하여 여러 단과대 캠퍼스 중 가장 번화가와 근접합니다. 그렇다고 밋밋한 현대식 건물이 들어서있는 것은 아닙니다. 코펜하겐 도시는 1,200년 된 건물들을 그대로 간직해 유럽 전통식 건물형태를 띠어 한국과 다른 분위기의 캠퍼스 생활을 즐길 수 있습니다. \n그리고 코펜하겐은 유럽 중심부는 아니지만 북유럽을 들어서는 관문지역에 있기에 다른 유럽지역과의 왕래가 편리합니다. 저가항공의 취항률이 높아 여행가기 무척 유리하니 교환학생 기간 동안 이점을 잘 활용하시길 추천합니다. \n\n코펜하겐은 위도 북쪽에 위치하여 여름에는 낮이 길고, 겨울에는 밤이 긴 특징을 가지...","제가 공부하였던 사회과학대 캠퍼스(CSS)는 위에서 언급했다시피 도시 중심에 위치하여 번화가와 매우 근접합니다. 도서관, 서점, 카페, 쇼핑가, 마켓 등등 시설이 모두 근접하여 물건 조달이 매우 쉽습니다. 교통편도 버스, 지하철, 기차 등 모두 통하여 시내를 온다면 이 곳을 지나치지 않을 수 없습니다. 또한 캠퍼스 바로 앞에는 코펜하겐 대학 소속 실내외 식물원이 있는데 여름에 산책을 하시길 추천합니다.","저는 첫 학기에 국제처에서 offer를 받은 학생 레지던스에서 살았습니다. 제가 다니던 캠퍼스 근처에 있는 한 일반 건물 전체가 학생 레지던스였는데, 한 층에 각 방으로 5명이 함께 살며 부엌 1실과 화장실이 2개를 공유하는 형식이었습니다. 그 레지던스가 모두 교환학생이 지내는 곳이라 항상 교환학생의 장이 되어 많은 친구들을 사귈 수 있는 기회가 주어져 한 학기가 매우 즐거웠습니다. 그리고 인터넷을 통해 집을 구하는 방법이 있어 2학기에는 따로 집을 구해 덴마크 친구들이랑 살게 되었습니다. 덴마크 가정집에서 덴마크 친구들과 지내는 생활도 현지 문화를 아는데 매우 유익하였습니다.","저는 경제학과 수업을 들었는데 영어로 된 수업 대부분은 석사과정 수업이었습니다. 코펜하겐 대학 경제학과의 시스템에서는 경제의 기초과목인 “미시경제원론”, ”거시경제원론”, ”미시경제학”, ”거시경제학” 과목들을 학사과정 3년 동안 배우며 경제의 기초를 확실히 다진 후 응용과목들을 석사과정에 포함시켜 수업을 진행합니다. 이런 시스템으로 우리나라 경제학과 학생들이 당황할 수 있으니 유의하시길 바랍니다. 그러나 교수진들은 영어권 교수님들을 포함하여 각국 교수님들의 유창한 영어실력을 갖고 계셔 수업전달능력은 문제가 없었습니다. \n경제학과의 시험의 경우 대부분이 필기시험입니다. 이 필기는 컴퓨터로 진행되어 학기 초에 컴퓨터 시험 사용법 특강을 하고 있으니 학기 초에 유념하시고 특강신청을 꼭 하시길 바랍니다. 우리나라와 다른 시스템으로 진행되어 시험시간에 당황하지 않기 위해서 알아야 합니다. \n\n도서관도 매 캠퍼스 마다 해당 도서관이 있는데, 사회과학대 캠퍼스의 도서관은 캠퍼스 내...",코펜하겐 대학의 국제처는 시내에 위치하고 있습니다. 국제처의 업무시간은 평일 12시부터 3시까지인 매우 짧은 시간이기 때문에 이런 점을 유의하여 방문하셔야 합니다. 학기 초에는 많은 교환학생들이 현지 생활 적응을 위해 많은 문의를 하여 국제처에 사람이 많을 테니 줄 서서 기다리시는 것을 염두 해 두셔야 합니다. 보통 혼잡한 시간은 점심시간이니 2시 정도에 가시길 권장합니다. 또한 국제처에서 학기 중 자주 Field Trip을 주체하는 데 이러한 기회를 놓치지 말고 참여하시길 추천합니다. 보통 Field Trip을 덴마크 내 다른 도시로 가는 데 교통비가 비싼 덴마크에서는 혼자 가면 매우 큰 부담이 되기에 국제처를 통해서 가면 다른 교환학생들과 교류도 생기고 저렴한 비용으로 덴마크 타도시의 방문이 가능합니다.,"공식적인 동아리의 존재여부는 잘 모르겠습니다. 그러나 경제학과에는 멘토프로그램이 잘 되있어, 매 학기 경제학과의 모든 멘토와 멘티들이 같이 활동하는 경우가 많습니다. 이런 프로그램을 잘 활용하면 교환학생 뿐만 아니라 현지학생들과의 교류가 빈번해져 좋은 경험으로 남습니다. 경제학과 학생은 멘토를 받는 것을 강력추천 합니다. \n사회과학대 내 컴퓨터실이 많습니다. 각 컴퓨터실 마다 각각의 단과 소속이지만 모든 학생들이 이용가능 합니다. 그러나 경제학과 컴퓨터실에서 ID를 등록하면 프린터 계정을 받아 100장 정도 프린트를 공짜로 사용한 후 그 뒤부터는 충전하여 사용할 수 있습니다.","덴마크 생활을 하면서 제가 받은 가장 큰 문화충격은 행복에 관한 가치관이었습니다. 여기 사람들의 행동, 생활에서는 전혀 쫓김, 경쟁 등이 없었습니다. 항상 편안한 얼굴로 자신의 여유를 즐기는 듯한 인상을 받았어요. 이들의 인생에서 가장 중요한 건 자신의 행복이었기 때문이죠. 자기가 가장 좋아하고 즐길만한 것을 찾아서 하고 가족들과 화목한 관계를 맺는 것이 그들에게 일순위였어요. 다른 사람들과의 경쟁에서 이기는 것이 목표가 아닌거죠. 그래서 타인에게 편안한 인상을 심어줄 수 있었던 것입니다. 이런 여유에서 저는 문화충격을 받았습니다. 그러나 이 문화충격을 통해 저 또한 행복에 대해 다시 생각하게 되고 저 자신만의 행복을 찾으려는 시도를 하게 되었습니다. 처음에는 “왜 모르는 사람이 나한테 웃지?”라는 생각을 하실 텐데, 그들의 미소에는 이러한 행복에 대한 가치관이 숨겨져 있답니다.","가장 친절하게 도움을 주시는 곳은 ''한국센터''와 대사관이 있겠습니다. 한국센터는 한인 목사님이 운영하시는 곳인데요 한인관광객들에게 저렴한 비용으로 숙소를 제공해주시기도 하고, 각 분기별로 유학생 삼겹살 파티 등의 모임도 개최하여 주십니다. 목사님이 김치를 직접 담가서 저희 삼겹살 파티 때 제공해 주십니다. 대사관 또한 민사-행정적으로 문제가 생겼을 땐, 적극적으로 도움을 주십니다. 재외국민 등록을 하면 덴마크 내 뿐 만 아니라 유럽 어느 지역에서 문제가 생겨도 영사 내 국민이기에 큰 도움을 주십니다. 그래서 제때 도움을 받기 위해서는 재외국민 등록을 하시길 바랍니다.","코펜하겐은 한국에 많이 알려지지 않은 나라입니다. 우유나 안데르센의 동화만 생각하시지요. 하지만 코펜하겐은 우리나라사람들이 생각하는 것보다 다채로운 나라입니다. 디자인 강국으로 색다른 디자인 제품, 건물 등 도시 곳곳에서 창의적인 디자인을 엿볼 수 있으며, 영화제, 재즈페스티벌, 전시, 각종 회의 등등 다양한 국제적인 행사가 이루어지고 있습니다. 도시 내에서 이러한 행사들이 진행되어 직접 체험할 수 있는 기회가 많습니다. 어떤 행사가 진행되는 지 사전에 몰랐다 하더라도 도시에 들어서는 순간 이미 자기자신도 모른 채 행사의 일원으로 참여하고 있을 것입니다. 한국에서 경험할 수 없는 이런 경험은 계속 소중한 자산으로 남을 것입니다. \n코펜하겐은 저에게 행복관, 다양한 경험을 제공한 나라입니다. 여러분도 세계에서 가장 행복한 나라, 덴마크에서 다양한 경험을 통해 행복을 찾으시길 바랍니다."
65,"코펜하겐은 덴마크의 수도이며, 비록 규모는 크지 않지만, 세계적인 국제회의 개최지로 10위 안에 드는 멋진 곳입니다. 유구한 역사를 가지고 있는 코펜하겐 대학의 건물은 도시 곳곳에 단과대학 별로 또는 각 강의실별로 흩어져 있습니다. 경제학과의 경우, 강의실이 열 개 정도의 작은 건물에 사이좋게 나누어져 있습니다. 처음에는 낯설었지만, 시간이 지날 수록 그 나름의 특유한 매력을 느꼈습니다.\n\n기후는 대체로 온화한 편입니다. 겨울에도 영하로 내려가는 날이 그다지 많지 않았습니다. 한 여름엔 보통 낮기온 20도를 전후로 덥지 않습니다. 다만, 겨울이 4~5월 까지 지속 되기 때문에, 따스한 봄을 맞으려면 기다림이 필요합니다. 저는 난방이 잘되는 곳에서 지내서 겨울에도 반팔을 입고 지냈지만, 몇몇 친

In [24]:
df_sample.columns.to_list()

['gen_info',
 'env_info',
 'food_info',
 'study_info',
 'office_info',
 'facil_info',
 'mhct_info',
 'help_info',
 'etc_info']

In [28]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_string.html

# combine column values into one string
str_gen_info = df_sample["gen_info"].to_string(index=False)

# replace line breaks
str_gen_info = str_gen_info.replace("\n", "")
str_gen_info = str_gen_info.replace("\\n", "")

# check number of letters
print(len(str_gen_info))
str_gen_info

35900


낮기는 하지만, 크게 우려 할 정도의 추위는 아닙니다. 겨울에 해가 오전 8시에 떠서, 오후 3~4시에 진다는 것도 큰 특징입니다. 나름의 운치가 있습니다.                                                                                                                                                                                                                                  코펜하겐 대학교는 연세대학교처럼 캠퍼스가 크지 않고  문과대학, 사회과학 대학, 이과대학 등이 나누어져 작은 캠퍼스들이 각기 다른 곳에 위치해 있습니다. 심리학과가 위치한 사회과학 대학은 코펜하겐의 중심지인 nørreport 에 위치해 있습니다.기후는 바람이 많고, 겨울이 매우 어둡습니다. 여름에는 밤 열한시 까지도 하늘이 밝고, 반면에 겨울에는 해가 오후 4시에 집니다. 겨울의 온도 자체는 의외로 서울보다 높은 편 이어서, 제가 있었던 해에는 눈이 별로 많이 내리지 않았습니다. 안녕하세요. 교환학생을 고민하신다면 저는 덴마크를 강! 추! 해드리고 싶습니다. 너무 아름다운 추억이 많은 일년이었네요! 먼저 코펜하겐 대학교는 캠퍼스가 네개로 나뉘어져 있고요. 연세대학교 캠퍼스에 비하면 작은 크키입니다. 학생수 부터가 작고요. 하지만 아담 아담한 매력이 있어요. 경제학과 였던 저는 시티 켐퍼스 CSS 에서 주로 수업을 들었고요, 학기 시작전에 대부분의 학생들이 수강하는 pre danish course 는 Amager 캠퍼스에서 듣게 될것이에요. CSS는 시내 중심인 Norreport역과 매우 가까이 위치해서 좋아요. 또 건물들이 고풍스럽게 예쁩니다. 약간 낡긴 했지만요 (연희관과 비슷) 그리고 아마 캠퍼스는 새건물이라 도서관이나 교실이나 시설이 좋구요, 칸틴(학생식당) 매우 좋습니다. 다른 캠퍼스는 공부해 보지 않아